Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable use of significant whitespace. Its language constructs and object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.

Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. Python is often described as a "batteries included" language due to its comprehensive standard library.

<sup id="fn1">[src]</sup><a href="http://cpython-devguide.readthedocs.io/compiler/">Design of CPython’s Compiler</a><br/>
<sup id="fn1">[src]</sup> <a href="http://pgbovine.net/cpython-internals.htm">Philip Guo CPython internals lectures</a><br/>
<sup id="fn1">[src]</sup> <a href="http://trizpug.org/Members/cbc/wyntkap/compiler.html" title="PyOhio PyCamp 2014 :: The Compiler">PyOhio PyCamp 2014 :: The Compiler</a><br/>
<sup id="fn1">[src]</sup> <a href="https://greentreesnakes.readthedocs.io" title="The missing Python AST docs">The missing Python AST docs</a><br/>
<sup id="fn1">[src]</sup> <a href="pythontutor.com" title="Visualize program execution">Visualize program execution</a><br/>

# High-level

In computer science, a high-level programming language is a programming language with strong abstraction from the details of the computer. In contrast to low-level programming languages, it may use natural language elements, be easier to use, or may automate (or even hide entirely) significant areas of computing systems (e.g. memory management), making the process of developing a program simpler and more understandable than when using a lower-level language

[*Wikipedia*](https://en.wikipedia.org/wiki/High-level_programming_language)

# Interpreted

There are three general modes of execution for modern high-level languages:

- **Interpreted**: the syntax is read and then executed directly, with no compilation stage. A program called an interpreter reads each program statement, following the program flow, then decides what to do, and does it.

- **Compiled**: the code written in a language is compiled, its syntax is transformed into an executable form before running. There are two types of compilation:
    - **Machine code generation**
    - **Intermediate representations**: the code written in a language is compiled to an intermediate representation, that representation can be optimized or saved for later execution without the need to re-read the source file. When the intermediate representation is saved, it may be in a form such as bytecode. The intermediate representation must then be interpreted or further compiled to execute it.

- **Source-to-source translated or transcompiled**

# Code structure

The code directory structure is described in <a href="http://cpython-devguide.readthedocs.io/setup/#directory-structure">Python Developer's Guide</a>. Guido van Rossum summarizes it in <a href="https://paper.dropbox.com/doc/Yet-another-guided-tour-of-CPython-XY7KgFGn88zMNivGJ4Jzv">Yet another guided tour of CPython</a>

- Include — header files
- Objects — object implementations, from int to type
- Python — interpreter, bytecode compiler and other essential infrastructure
- Parser — parser, lexer and parser generator
- Modules — stdlib extension modules, and `main.c` 
- Programs — not much, but has the real `main()` function

In CPython, the compilation from source code to bytecode involves several steps <sup id="fn1"><a href="http://cpython-devguide.readthedocs.io/compiler/">[src]</a></sup>

- Parse source code into a parse tree (Parser/pgen.c)
- Transform parse tree into an Abstract Syntax Tree (Python/ast.c)
- Transform AST into a Control Flow Graph (Python/compile.c)
- Emit bytecode based on the Control Flow Graph (Python/compile.c)

### AST

[Doc](https://docs.python.org/3/library/ast.html)

The code is parsed, i.e. split up into a list of pieces called tokens. These tokens are based on a set of rules for things that should be treated differently. For instance, the keyword *if* is a different token than a numeric value like *42*.

The list of tokens is transformed to build an **Abstract Syntax Tree, AST**, collection of nodes which are linked together based on the Python language [grammar](https://docs.python.org/3/reference/grammar.html).

In python, everything is an object, AST represents logicaly each element as an object. [A third pary documentation on AST](https://greentreesnakes.readthedocs.io)

In [10]:
import ast
from ast import PyCF_ONLY_AST

In [122]:
# The code is beeing striped to remove
# left and right space before parsing
# and limit thee size of the tree
code = """
def hello(who: str) -> None:
    msg = f'Hello {who}'
    print(msg)

hello("world")
"""

code = code.strip()

Execute the code

In [88]:
exec(code)

Hello world


In [89]:
tree = ast.parse(code)

The code is seen as two elements in the tree

In [102]:
for i, elt in enumerate(tree.body):
    print(i, elt)

0 <_ast.FunctionDef object at 0x7f79c9aa0690>
1 <_ast.Expr object at 0x7f79c9ad1890>


In [104]:
# Get the FunctionDef
fdef = tree.body[0]
# Get the first function argument
arg = fdef.args.args[0]

Some informations from the tree can be retrieved:

In [110]:
print(f'{arg.arg}: {arg.annotation.id} at col #{arg.col_offset}')

who: str at col #10


The code can be compiled

In [112]:
lines = [None] + code.splitlines()  # None at [0] so we can index lines from 1
test_namespace = {}

for node in tree.body:
    wrapper = ast.Module(body=[node])
    try:
        co = compile(wrapper, "<ast>", 'exec')
        exec(co, test_namespace)
    except AssertionError as e:
        print("Assertion failed on line", node.lineno, ":")
        print(lines[node.lineno])
        # If the error has a message, show it.
        if e.args:
            print(e)
        print()

Hello world


In [116]:
code = compile(tree, filename="<ast>", mode="exec")

In [120]:
class FuncLister(ast.NodeVisitor):
    def visit_FunctionDef(self, node):
        print(node.name)
        self.generic_visit(node)

FuncLister().visit(tree)

hello


### Bytecode

From an abstract syntax tree, the interpreter can produce a lower level form of instructions called **bytecode**. These instructions are things like `BINARY_ADD` and are meant to be very generic so that a computer can run them.

In [1]:
print("Hello, World!")

Hello, World!


In [2]:
from dis import dis
dis('print("Hello, World!")')

  1           0 LOAD_NAME                0 (print)
              2 LOAD_CONST               0 ('Hello, World!')
              4 CALL_FUNCTION            1
              6 RETURN_VALUE


CPython uses a stack-based virtual machine. That is, it's oriented entirely around stack data structures (where you can "push" an item onto the "top" of the structure, or "pop" an item off the "top").

With the bytecode instructions available, the interpreter can finally run your code. The bytecode is used to call functions in your operating system which will ultimately interact with a CPU and memory to run the program.

# Opcodes and main interpreter loop

In [126]:
import tempfile
from os.path import join as pjoin

tmp = tempfile.gettempdir()
fName = pjoin(tmp, 'test.py')

codeExemple = """ppl = ['Alice', 'Bob', 'Carol', 'Doug']
excited_ppl = [e + '!!' for e in ppl]
ppl_len = [len(x) for x in ppl]"""


with open(fName, 'w') as f:
    f.write(codeExemple)

In [127]:
!python -m dis {fName}

  1           0 LOAD_CONST               0 ('Alice')
              2 LOAD_CONST               1 ('Bob')
              4 LOAD_CONST               2 ('Carol')
              6 LOAD_CONST               3 ('Doug')
              8 BUILD_LIST               4
             10 STORE_NAME               0 (ppl)

  2          12 LOAD_CONST               4 (<code object <listcomp> at 0x7f43f9dc4ed0, file "/tmp/test.py", line 2>)
             14 LOAD_CONST               5 ('<listcomp>')
             16 MAKE_FUNCTION            0
             18 LOAD_NAME                0 (ppl)
             20 GET_ITER
             22 CALL_FUNCTION            1
             24 STORE_NAME               1 (excited_ppl)

  3          26 LOAD_CONST               6 (<code object <listcomp> at 0x7f43f9dca390, file "/tmp/test.py", line 3>)
             28 LOAD_CONST               5 ('<listcomp>')
             30 MAKE_FUNCTION            0
             32 LOAD_NAME                0 (ppl)
             34 GET_ITER
             

In [128]:
c = compile(codeExemple, 'test.py', 'exec')

In [129]:
c.co_code

b'd\x00d\x01d\x02d\x03g\x04Z\x00d\x04d\x05\x84\x00e\x00D\x00\x83\x01Z\x01d\x06d\x05\x84\x00e\x00D\x00\x83\x01Z\x02d\x07S\x00'

In [130]:
!cat {fName}

ppl = ['Alice', 'Bob', 'Carol', 'Doug']
excited_ppl = [e + '!!' for e in ppl]
ppl_len = [len(x) for x in ppl]

## opcode.h

In [131]:
!wget --quiet https://raw.githubusercontent.com/python/cpython/master/Include/opcode.h -O {tmp}/opcode.h
!head -n 30 {tmp}/opcode.h

/* Auto-generated by Tools/scripts/generate_opcode_h.py from Lib/opcode.py */
#ifndef Py_OPCODE_H
#define Py_OPCODE_H
#ifdef __cplusplus
extern "C" {
#endif


    /* Instruction opcodes for compiled code */
#define POP_TOP                   1
#define ROT_TWO                   2
#define ROT_THREE                 3
#define DUP_TOP                   4
#define DUP_TOP_TWO               5
#define ROT_FOUR                  6
#define NOP                       9
#define UNARY_POSITIVE           10
#define UNARY_NEGATIVE           11
#define UNARY_NOT                12
#define UNARY_INVERT             15
#define BINARY_MATRIX_MULTIPLY   16
#define INPLACE_MATRIX_MULTIPLY  17
#define BINARY_POWER             19
#define BINARY_MULTIPLY          20
#define BINARY_MODULO            22
#define BINARY_ADD               23
#define BINARY_SUBTRACT          24
#define BINARY_SUBSCR            25
#define BINARY_FLOOR_DIVIDE      26
#define BINARY_TRUE_DIVIDE       27


## ceval.c

In [133]:
!wget --quiet https://raw.githubusercontent.com/python/cpython/master/Python/ceval.c -O {tmp}/ceval.c
!head -n 20 {tmp}/ceval.c


/* Execute compiled code */

/* XXX TO DO:
   XXX speed up searching for keywords by using a dictionary
   XXX document it!
   */

/* enable more aggressive intra-module optimizations, where available */
#define PY_LOCAL_AGGRESSIVE

#include "Python.h"
#include "pycore_call.h"
#include "pycore_ceval.h"
#include "pycore_code.h"
#include "pycore_object.h"
#include "pycore_pyerrors.h"
#include "pycore_pylifecycle.h"
#include "pycore_pystate.h"
#include "pycore_tupleobject.h"


<iframe width="800" height="500" frameborder="0" src="http://pythontutor.com/iframe-embed.html#code=x+%3D+5%0Ay+%3D+10%0Az+%3D+x+%2B+y&cumulative=false&py=2&curInstr=3"/>

# General purpose

[https://awesome-python.com/](awesome-python):

___
References:
- [Your Guide to the CPython Source Code](https://realpython.com/cpython-source-code-guide/)
- [Inside The Python Virtual Machine](https://leanpub.com/insidethepythonvirtualmachine/read)
- [An introduction to Python bytecode](https://opensource.com/article/18/4/introduction-python-bytecode)
- [Deciphering Python: How to use Abstract Syntax Trees (AST) to understand code
](https://www.mattlayman.com/blog/2018/decipher-python-ast/)

# Garbage collected

https://github.com/python/cpython/blob/ce6a070414ed1e1374d1e6212bfbff61b6d5d755/Include/object.h#L104

In [134]:
from sys import getrefcount

In [135]:
a = "un"

In [136]:
getrefcount(a)

2

In [137]:
b = a

In [138]:
b is a

True

In [139]:
getrefcount(a)

3

In [140]:
del(b)

In [141]:
getrefcount(a)

2